In [1]:
import sys
from pathlib import Path
from stable_baselines3.dqn import DQN
import optuna

PROJECT_ROOT_DIR = Path().absolute().parent.parent
if str(PROJECT_ROOT_DIR.absolute()) not in sys.path:
    sys.path.append(str(PROJECT_ROOT_DIR.absolute()))

from envs.sys_id_env import SystemIdentificationEnv
from utils.wrappers.multibinary_to_discrete import MultiBinaryToDiscreteWrapper

optuna.logging.set_verbosity(optuna.logging.WARNING)  # 关闭Optuna控制台的输出

PROJECT_ROOT_DIR

/home/ucav/anaconda3/envs/new_gymnasium/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PosixPath('/home/ucav/pythonprojects/rl_sys_id')

In [ ]:
params_config = {
    "g": {
        "initial_value": 10.0,
        "optimize": True,
        "range": [9.0, 10.0],
    },
    "m": {
        "initial_value": 1.0,
        "optimize": True,
        "range": [0.8, 1.2],
    },
    "l": {
        "initial_value": 1.0,
        "optimize": True,
        "range": [0.7, 1.3],
    },
}

env = SystemIdentificationEnv(
    dynamics_env_id="CustomPendulum-v0",
    params_config=params_config,
    obs_real_file_path=PROJECT_ROOT_DIR / "data/custom_pendulum/permanent/obs_real.npy",
    act_real_file_path=PROJECT_ROOT_DIR / "data/custom_pendulum/permanent/act_real.npy",
    next_obs_real_file_path=PROJECT_ROOT_DIR / "data/custom_pendulum/permanent/next_obs_real.npy",
    bo_optimizer_n_trials=30,
    bo_optimizer_n_jobs=16,
    bo_optimizer_sample_num_in_optimize=1000,
    reward_b=0.2,
    max_steps=10,
    loss_threshold=1e-10,
)
env = MultiBinaryToDiscreteWrapper(env)

/home/ucav/anaconda3/envs/new_gymnasium/lib/python3.12/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/ucav/anaconda3/envs/new_gymnasium/lib/python3.12/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [3]:
policy_path = PROJECT_ROOT_DIR / "checkpoints/sys_id/custom_pendulum/sac/g_9_5_m_0_9_l_1_2/seed_2/best_model.zip"
algo = DQN.load(policy_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [4]:
obs, info = env.reset(seed=17954)
print(f"info 0: {info}")

while True:
    action, _ = algo.policy.predict(observation=obs, deterministic=True)
    next_obs, reward, terminated, truncated, info = env.step(action=action)
    print(obs, action, reward, next_obs)
    print(info)
    obs = next_obs
    if terminated or truncated:
        print("finished")
        break

info 0: {'loss': 0.0018601656}
[10.  1.  1.] 4 -0.18693282714061368 [9.003447 1.       1.141486]
{'params_before_optimize': {'g': 10.0, 'm': 1.0, 'l': 1.0}, 'params_selected_to_optimize': {'g': {'initial_value': 10.0, 'optimize': True, 'range': [9.0, 10.0]}, 'l': {'initial_value': 1.0, 'optimize': True, 'range': [0.7, 1.3]}}, 'params_after_optimize': {'g': 9.00344678715477, 'l': 1.141486066467132}, 'loss': 4.245995057772234e-07, 'success': False}
[9.003447 1.       1.141486] 1 -0.2546968811633697 [9.003447  0.9771619 1.141486 ]
{'params_before_optimize': {'g': 9.00344678715477, 'm': 1.0, 'l': 1.141486066467132}, 'params_selected_to_optimize': {'m': {'initial_value': 1.0, 'optimize': True, 'range': [0.8, 1.2]}}, 'params_after_optimize': {'m': 0.9771619088684472}, 'loss': 1.9937456272600684e-06, 'success': False}
[9.003447  0.9771619 1.141486 ] 1 -0.22591209408838342 [9.003447  1.0072787 1.141486 ]
{'params_before_optimize': {'g': 9.00344678715477, 'm': 0.9771619088684472, 'l': 1.1414860